In [2]:
import sys
for entry in sys.path:
    if 'python2.7' in entry:
        sys.path.remove(entry)

In [3]:
sys.path.remove('/usr/local/lib/python2.7/site-packages/mxnet-0.9.1-py2.7-macosx-10.11-x86_64.egg')

In [4]:
import tensorflow as tf
import numpy as np
import matplotlib.image as mpimg
import os
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from numpy import array
import pickle

def get_training_images():
    training_images = ['train_data/{:05d}.jpg'.format(i) for i in range(1, 25113)]
    training_images = [np.float32(mpimg.imread(img))/255 for img in training_images]
    training_imgs = [img.flatten() for img in training_images]
    training_imgs = [np.append(img, 1.0) for img in training_imgs]
    return np.float32(np.array(training_imgs))

def get_testing_images():
    testing_images = ['test_data/{:04d}.jpg'.format(i) for i in range(1, 4983)]
    testing_images = [np.float32(mpimg.imread(img))/255 for img in testing_images]
    testing_imgs = [img.flatten() for img in testing_images]
    testing_imgs = [np.append(img, 1.0) for img in testing_imgs]
    return np.float32(np.array(testing_imgs))

def get_training_label():
    with open('labels/train_label.txt', 'r') as training_labels:
        training_labels = training_labels.readlines()
        new_training_labels = []
        for i in training_labels:
            new_training_labels.append(int(i.split('\n')[0]))
        training_values = array(new_training_labels)
        onehot_encoder = OneHotEncoder()
        label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(training_values)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    return np.float32(onehot_encoder.fit_transform(integer_encoded).toarray())

def get_testing_label():
    with open('labels/test_label.txt', 'r') as testing_labels:
        testing_labels = testing_labels.readlines()
        new_testing_labels = []
        for i in testing_labels:
            new_testing_labels.append(int(i.split('\n')[0]))
        testing_values = array(new_testing_labels)
        onehot_encoder = OneHotEncoder()
        label_encoder = LabelEncoder()
        integer_encoded = label_encoder.fit_transform(testing_values)
        integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    return np.float32(onehot_encoder.fit_transform(integer_encoded).toarray())

/Applications/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [5]:
learning_rate = 0.01
training_epochs = 20
batch_size = 100
display_step = 1

# tf Graph Input
x = tf.placeholder(tf.float32, [None,785]) # mnist data image of shape 28*28=784
y = tf.placeholder(tf.float32, [None, 5]) # 0-9 digits recognition => 10 classes

# Set model weights
W = tf.Variable(tf.random_uniform([784, 5]))
b = tf.Variable(tf.ones([1, 5]))
theta = tf.Variable(tf.concat([W, b], 0), dtype=tf.float32)
#theta = tf.cast(theta, dtype=tf.float32)
theta.shape

TensorShape([Dimension(785), Dimension(5)])

In [1]:
softmax_logits = tf.nn.softmax(tf.matmul(tf.cast(x, tf.float32), tf.cast(theta, tf.float32)))
        #print third_part
negative_one = tf.constant(np.float32(-1))
inner_subtraction = tf.subtract(y, softmax_logits)
gradient = tf.matmul(x, inner_subtraction, transpose_a = True)
gradient = tf.scalar_mul(negative_one, gradient)
new_theta = tf.assign_sub(theta, tf.scalar_mul(learning_rate, gradient))

# Construct model
#pred = tf.nn.softmax(tf.matmul(x, theta)) # Softmax

# Minimize error using cross entropy
#cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
# Gradient Descent
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

NameError: name 'tf' is not defined

In [ ]:
# Start training
training_error = []
testing_error = []
with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch in range(training_epochs):

        train_data, train_labels = get_training_images(), get_training_label()
            # Fit training using batch data
        sess.run(new_theta, feed_dict={x: train_data, y: train_labels})
            # Compute average loss
            #avg_cost += c / total_batch
        # Display logs per epoch step
        #if (epoch+1) % display_step == 0:
            #print "Epoch:", '%04d' % (epoch+1), "theta=", "{:.9f}".format(theta)
        train_output = tf.matmul(train_data, theta)
        train_output_softmax = tf.nn.softmax(train_output)
        max_values_train = tf.argmax(train_output_softmax, axis = 1)
        training_differences = tf.equal(tf.cast(max_values_train, tf.float32), tf.cast(tf.argmax(train_labels, axis=1), tf.float32))
        training_accuracy_rate = tf.reduce_mean(tf.cast(training_differences, tf.float32))
        training_error.append(sess.run(1 - training_accuracy_rate))
        print(training_error)
        
        test_data = get_testing_images()
        testing_labels = get_testing_label()
        
        test_output = tf.matmul(test_data, theta)
        test_output_softmax = tf.nn.softmax(test_output)
        max_values_test = tf.argmax(test_output_softmax, axis = 1)
        differences = tf.equal(tf.cast(max_values_test, tf.float32), tf.cast(tf.argmax(testing_labels, axis=1), tf.float32))
        
        prediction = np.asarray(sess.run(max_values_test))
        print(prediction)
        testing_accuracy_rate = tf.reduce_mean(tf.cast(differences, tf.float32))
        testing_error.append(sess.run(1 - testing_accuracy_rate))
        print(testing_error)
        testing_accuracy_rate = sess.run(testing_accuracy_rate)
        if testing_accuracy_rate > 0.92:
            np.savetxt("prediction.csv", prediction, delimiter=",")
    
        
    import pickle
    final_theta = sess.run(theta)
    filehandler = open("multiclass_parameters.txt","wb")
    pickle.dump(final_theta, filehandler)
    filehandler.close()   
    
    import matplotlib.pyplot as plt
    for i in range(0, 5):
        W_k = final_theta[0:784,i]
        Img = W_k.reshape(28,28)
        plt.savefig("Digits_Images.png")
        plt.colorbar()
        plt.show()

[0.70687318]
[0 0 0 ..., 0 2 2]
[0.70232838]
[0.70687318, 0.60576618]
[1 1 4 ..., 1 1 4]
[0.70232838, 0.58952224]
[0.70687318, 0.60576618, 0.66784805]
[2 2 2 ..., 2 2 2]
[0.70232838, 0.58952224, 0.65455639]


In [ ]:
plt.subplot(2,1,1)
plt.plot(training_error, '-o', label='Training_Error')
plt.title('Training Error')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend(ncol=2, loc='upper right')
plt.subplot(2,1,2)
plt.plot(testing_error, '-o', label='Testing_Error')
plt.title('Testing Error')
plt.xlabel('Epoch')
plt.ylabel('Error')
plt.legend(ncol=2, loc='lower right')
plt.gcf().set_size_inches(10, 10)
plt.savefig('Training_and_testing_error.png')